<h1 style="color:#2E86C1; font-size: 36px; text-align: center;">Código fuente - TFG Matemáticas </h1>
<h2 style="color:#117864; font-size: 24px; text-align: center;">Una aproximación MILP a la optimización del trazado de redes de metro en entornos urbanos</h2>

<br>

<div style="text-align: center; font-size: 18px;">
  <strong>Autora:</strong> Nuria Pedrosa Ortigosa
  <br>
  <strong>Titulación:</strong> Doble Grado Matemáticas e Ingeniería Informática
  <br>
  <strong>Universidad:</strong> Universidad de Málaga (Facultad de Ciencias)
  <br>
  <strong>Año:</strong> 2025
</div>

<hr style="margin-top: 30px; margin-bottom: 30px;">


# **IMPORTACIÓN DE LIBRERÍAS** 

In [ ]:
import numpy as np
from scipy.optimize import linprog
from scipy.sparse import lil_matrix
from scipy.sparse import vstack
import matplotlib.pyplot as plt
import re
import pulp

# **REPRESENTACIÓN DE LA CIUDAD** 

Recordamos que las variables principales del problema son $X_{i,j,k,l,m}$ con $(i,j,k,l) \in Conexiones\_validas$ y $m \in \{1,...,M\}$, siendo el tablero de dimensión $N\times N$, $M$ el número de líneas de metro a construir y $Conexiones\_validas$ un conjunto a determinar según citerios más tarde justificados. Serán binarias y tomarán valores de la siguiente forma:

$ X_{i,j,k,l,m} = \begin{cases} 1 \text{ si el punto } (i,j) \text{ se conecta al punto } (k,l) \text{ mediante la línea } m & \\ 0 \text{ cc}
  \end{cases} $

Además, con la información previa proporcionada por la distribución del tablero, podrán computarse las siguientes matrices:

$ E_{i,j} = \begin{cases} 1 \text{ si el punto } (i,j) \text{ es estación} & \\ 0 \text{ cc}
  \end{cases} $

$  I_{i,j} = \begin{cases} 1 \text{ si el punto } (i,j) \text{ es estación de interés } & \\ 0 \text{ cc}
   \end{cases} $

$ R_{i,j} = \begin{cases} 1 \text{ si el punto } (i,j) \text{ está en la mitad superior y/o izquierda del río } & \\ 0 \text{ cc}
  \end{cases} $

$ D_{i,j} = \text{ nº de distrito del punto } (i, j) $


$ E_{i,j},  I_{i,j}, R_{i,j}, D_{i,j} $, al indicar qué puntos son estaciones, estaciones de interés, a qué lado del río se encuentran y a qué distrito pertenecen, serán propias de cada configuración del tablero. Serán computables una vez se disponga de él. Sin embargo, el resto de matrices auxiliares, al depender únicamente de las coordenadas de los puntos, sí pueden calcularse a priori.

### CONFIGURACIÓN DEL MAPA Y ELECCIÓN DE PARÁMETROS ###

In [ ]:
# Tamaño tablero
N = 10
# Número de líneas
M = 4
# Número de distritos
num_distritos = 13
# Número mínimo de distritos visitados
min_distritos = 9
# Número mínimo de estaciones de interés a visitar
min_estaciones_especiales = 3
# Longitud máxima construible de vía
max_long = 60
# Cantidad de estaciones visitadas por cada línea
estaciones_por_linea = 10
# Estación central
estacion_central = (4, 4)


# E(i,j) indica si el punto (i,j) es una estación de metro
E = np.zeros((N,N), dtype = int)
estaciones = [
    (0, 0), (0, 1), (0, 2), (0, 4), (0, 5), (0, 7), (0, 9),
    (1, 1), (1,3), (1,6), (1,8),(1,9),
    (2, 0), (2, 3), (2, 6), (2, 9),
    (3, 0), (3, 2), (3, 4), (3, 5), (3, 6), (3, 7), (3, 9),
    (4, 1), (4, 2), (4, 4), (4, 5), (4, 8),
    (5, 0), (5, 2), (5, 4), (5, 7),
    (6, 3), (6, 4), (6, 6), (6, 7), (6, 9),
    (7, 0), (7, 2), (7, 3), (7, 5), (7, 8), (7, 9),
    (8, 1), (8, 6), (8, 8),
    (9, 0), (9, 1), (9, 3), (9, 4), (9, 5), (9, 7), (9, 9)
]
for (i, j) in estaciones:
    E[i, j] = 1


# I(i,j) indica si el punto (i,j) es una estación de interés
I = np.zeros((N,N), dtype = int)
importantes = [
    (1, 6), (3, 0), (3, 5), (6, 9), (9, 4), (0,0), (0,9), (9,0), (9,9)
]
for (i, j) in importantes:
    I[i, j] = 1


# R(i,j) indica si el punto (i,j) está en una u otra mitad del río
# (1 si está en la mitad superior izquierda, 0 al contrario)
R = np.zeros((N,N), dtype = int)
for i in range(N):
    for j in range(N):
      if 0<=i<4:
        R[i,j]=0
      elif 6<=i<10:
        R[i,j]=1
R[4,1]=R[4,2]=R[5,0]=R[5,2]=R[5,7]=1


# D(i,j) indica el distrito de cada punto (i,j)
D = np.zeros((N,N), dtype = int)
for i in range (N):
  for j in range (N):
    if 0<=i<3 and 0<=j<3:
      D[i,j] = 1
    elif 0<=i<3 and 3<=j<7:
      D[i,j] = 2
    elif 0<=i<3 and 7<=j<10:
      D[i,j] = 3
    elif 3<=i<7 and 0<=j<3:
      D[i,j] = 4
    elif 3<=i<7 and 3<=j<7:
      D[i,j] = 5
    elif 3<=i<7 and 7<=j<10:
      D[i,j] = 6
    elif 7<=i<10 and 0<=j<3:
      D[i,j] = 7
    elif 7<=i<10 and 3<=j<7:
      D[i,j] = 8
    elif 7<=i<10 and 7<=j<10:
      D[i,j] = 9
D[0,0] = 10
D[0,9] = 11
D[9,0] = 12
D[9,9] = 13


# Estaciones raíces de cada línea
raices = []
raices.append((1,1))
raices.append((3,4))
raices.append((6,7))
raices.append((9,3))

### DEFINICIÓN DE LOS CONJUNTOS DE CONEXIONES VÁLIDAS Y ESTACIONES ###

Podemos considerar únicamente como variables las que definan segmentos válidos, y asumir que el resto no se activan. Los válidos serán los que tengan por extremos nodos que sean, en efecto, estaciones, y además estos puntos sean distintos.


In [ ]:
def conexiones_estaciones():
    """
    Devuelve una lista de todas las conexiones válidas entre estaciones en la forma 
    de 4-uplas (i,j,k,l), donde (i,j) y (k,l) son coordenadas de estaciones.
    """
    
    conexiones_validas = []

    for i in range(N):
      for j in range(N):
        for k in range(N):
          for l in range(N):
            if E[i,j] and E[k,l] and (i,j) != (k,l):
              conexiones_validas.append((i,j,k,l))

    return conexiones_validas


In [ ]:
# Conjunto de conexiones válidas entre estaciones
conexiones_validas = conexiones_estaciones()

# Número de conexiones válidas, o sea, número de variables de decisión de tipo X
N_val = len(conexiones_validas)
N_val

También  será de utilidad acceder a los índices del tablero que son estaciones:

In [ ]:
def nodos_estaciones():
    """
    Devuelve una lista de todas las estaciones en la forma de 2-uplas (i,j).
    """

    nodos_estaciones = []

    for i in range(N):
      for j in range(N):
        if E[i,j]:
          nodos_estaciones.append((i,j))

    return nodos_estaciones


In [ ]:
# Conjunto de estaciones
estaciones = nodos_estaciones()

# Número de estaciones
n_estaciones = len(estaciones)
n_estaciones

### PRECÁLCULO DE MATRICES Y OTRAS ESTRUCTURAS ÚTILES PARA LA DEFINICIÓN DE RESTRICCIONES ###

In [ ]:
def computarDIST():
    """
    Devuelve una matriz de distancias euclídeas entre pares de estaciones.
    DIST[i,j,k,l] representa distancia entre las estaciones (i,j) y (k,l).
    """ 

    DIST = np.zeros((N, N, N, N))
    for i in range(N):
        for j in range(N):
            for k in range(N):
                for l in range(N):
                    DIST[i, j, k, l] = np.sqrt((i - k)**2 + (j - l)**2)
    return DIST

In [ ]:
DIST = computarDIST()

In [ ]:
def computarH():
    """
    Devuelve una matriz H de tamaño NxNxNxN, donde H[i,j,k,l] = 1 si los puntos 
    (i,j) y (k,l) están sobre la misma línea horizontal, y 0 al contrario.
    """
  
    H = np.zeros((N, N, N, N), dtype=int)
    for i in range(N):
        for j in range(N):
            for k in range(i,N):
                for l in range(N):
                    H[i, j, k, l] = H[k, l, i, j] = int(j == l)
    return H

In [ ]:
def computarV():
    """
    Devuelve una matriz V de tamaño NxNxNxN, donde V[i,j,k,l] = 1 si los puntos 
    (i,j) y (k,l) están sobre la misma línea vertical, y 0 al contrario.
    """

    V = np.zeros((N, N, N, N), dtype=int)
    for i in range(N):
        for j in range(N):
            for k in range(i,N):
                for l in range(N):
                    V[i, j, k, l] = V[k, l, i, j] = int(i == k)
    return V

In [ ]:
def computarDIAG():
    """
    Devuelve una matriz DIAG de tamaño NxNxNxN, donde DIAG[i,j,k,l] = 1 si los puntos 
    (i,j) y (k,l) están sobre la misma línea diagonal, y 0 al contrario.
    """
    DIAG = np.zeros((N, N, N, N), dtype=int)
    for i in range(N):
        for j in range(N):
            for k in range(i,N):
                for l in range(N):
                    DIAG[i, j, k, l] = DIAG[k, l, i, j]  =int(abs(i - k) == abs(j - l))

    return DIAG

In [ ]:
H = computarH()
V = computarV()
DIAG = computarDIAG()

In [ ]:
def computarINTER():
    """
    Devuelve una matriz INTER de tamaño NxNxNxN, donde INTER[i,j,k,l] = 1 si en la línea 
    que una los puntos (i,j) y (k,l) existe una estación intermedia, y 0 al contrario.
    """

    INTER = np.zeros((N,N,N,N), dtype=int)

    for x1 in range (N):
      for y1 in range (N):
        for x2 in range (x1,N): # Evitar cálculos dobles
          for y2 in range (N):

              if x1 == x2 and y1 == y2:
                continue  # Evitar cálculos para el mismo punto

              # Mismo eje x
              elif x1 == x2:
                  step = 1 if y1 < y2 else -1

                  for j in range(y1+step,y2,step):
                      if E[x1,j]:
                          INTER[x1,y1,x2,y2] = 1
                          INTER[x2,y2,x1,y1] = 1
                          # Salimos solo de este for
                          break

              # Mismo eje y
              elif y1 == y2:
                  step = 1 if x1 < x2 else -1

                  for i in range(x1+step,x2,step):
                      if E[i,y1]:
                          INTER[x1,y1,x2,y2] = 1
                          INTER[x2,y2,x1,y1] = 1
                          break

              # En diagonal
              elif abs(x1-x2) == abs(y1-y2):
                  x_step = 1 if x1 < x2 else -1
                  y_step = 1 if y1 < y2 else -1

                  x,y = x1,y1

                  while (x+step) != x2:
                      x += x_step
                      y += y_step

                      # Si permanece in bounds
                      if 0 <= x < N and 0 <= y < N:

                        if E[x,y]:
                            INTER[x1,y1,x2,y2] = 1
                            INTER[x2,y2,x1,y1] = 1
                            break
                      else:
                        break

              #El resto de casos no tiene interés estudiarlos, ya que si no
              #están en esas posiciones relativas, la restricción anterior
              #se encargaría de rechazar

    return INTER

In [ ]:
INTER = computarINTER()

In [ ]:
def es_direccion_valida(dx, dy):
    """
    Comprueba si la dirección (dx, dy) es válida, i.e., es horizontal, vertical o diagonal.
    Devuelve True si lo es, y False en caso contrario.
    """
    return (dx == 0 and dy != 0) or (dy == 0 and dx != 0) or (abs(dx) == abs(dy))



def normaliza_direccion(dx, dy):
    """
    Normaliza la dirección de un vector (dx, dy). 
    """

    if dx == 0:
        return (0, np.sign(dy))
    if dy == 0:
        return (np.sign(dx), 0)
    return (np.sign(dx), np.sign(dy))


pares_peligrosos = [
    # Horizontal vs vertical
    ((1, 0), (0, 1)), ((1, 0), (0, -1)), ((-1, 0), (0, 1)), ((-1, 0), (0, -1)),

    # Horizontal vs diagonales
    ((1, 0), (1, 1)), ((1, 0), (-1, 1)), ((1, 0), (1, -1)), ((1, 0), (-1, -1)),
    ((-1, 0), (1, 1)), ((-1, 0), (-1, 1)), ((-1, 0), (1, -1)), ((-1, 0), (-1, -1)),

    # Vertical vs diagonales
    ((0, 1), (1, 1)), ((0, 1), (-1, 1)), ((0, 1), (1, -1)), ((0, 1), (-1, -1)),
    ((0, -1), (1, 1)), ((0, -1), (-1, 1)), ((0, -1), (1, -1)), ((0, -1), (-1, -1)),

    # Diagonales perpendiculares (forman X)
    ((1, 1), (-1, 1)), ((1, 1), (1, -1)), ((1, -1), (-1, -1)), ((-1, 1), (-1, -1))
]


def segmentos_secantes(i,j,k,l):
    """
    Devuelve una lista de segmentos secantes al segmento de extremos (i,j) y (k,l), es decir, 
    aquellos que se cruzan con él en algún punto interior.
    Los segmentos se devuelven en la forma de 4-uplas (p,q,r,s), donde (p,q) y (r,s) los extremos.
    """
    segs_secantes = []

    for p, q, r, s in conexiones_validas:

        if p<i:
            # Para evitar repeticiones de restricciones
            continue

        denominador = (k - i) * (s - q) - (l - j) * (r - p)
        # Observación: denominador = 0 => segmentos paralelos

        # Comprobación de si están en la misma línea recta
        colineales = (q - j) * (k - i) == (l - j) * (p - i)


        # Estudiamos el caso de segmentos colineales
        if denominador == 0 and colineales:

            # Comprobamos si los segmentos se superponen en el eje x,
            # esto es, si las proyecciones de los puntos en el eje x se solapan
            x_overlap = max(i, k) >= min(p, r) and min(i, k) <= max(p, r)

            # Análogamente para el eje y
            y_overlap = max(j, l) >= min(q, s) and min(j, l) <= max(q, s)

            # Si ambos solapan, los segmentos se pisan en un tramo, 
            #como es el caso de la superposición parcial de (1,1)(1,3) y (1,2)(1,4)
            if x_overlap and y_overlap:
                segs_secantes.append((p, q, r, s))

        # En cualquier otro caso, comprobamos que no sean secantes en puntos interiores.
        # Solo son candidatos los perpendiculares (90º) y los que intersecan con (45º)
        if denominador != 0:

                dx1, dy1 = k - i, l - j
                dx2, dy2 = r - p, s - q

                if es_direccion_valida(dx1, dy1) and es_direccion_valida(dx2, dy2):

                        u = normaliza_direccion(dx1, dy1)
                        v = normaliza_direccion(dx2, dy2)

                        if (u, v) in pares_peligrosos or (v, u) in pares_peligrosos:

                                t_param = ((p - i) * (s - q) - (q - j) * (r - p)) / denominador
                                s_param = ((p - i) * (l - j) - (q - j) * (k - i)) / denominador

                                if 0 < t_param < 1 and 0 < s_param < 1:

                                    segs_secantes.append((p, q, r, s))

    return segs_secantes

# **DEFINICIÓN FORMAL DE RESTRICCIONES** #

### **RESTRICCIÓN TIPO 1: Longitud de vía construible** ###

Si llamamos $DIST_{i,j,k,l}$ a la distancia euclídea entre los puntos $(i,j)$ y $(k,l)$ en el plano, entonces, si $max\_long$ es la longitud máxima construible, debe ocurrir

$\sum_{(i,j,k,l) \in Conexiones\_validas,m \in \{1,...,M\} } DIST_{i,j,k,l,}X_{i,j,k,l,m} \leq max\_long$

De esta forma, si dos puntos no están conectados por una línea, como $X_{i,j,k,l,m}=0$, no afectan a la suma. En caso contrario, se suma la distancia que los une.

La matriz $DIST$ puede precalcularse fácilmente.

### **RESTRICCIÓN TIPO 2: número mínimo de estaciones de interés visitadas** ###

Se definen por necesidad variables binarias adicionales, $Y_{i,j,m}$ y $Z_{i,j}$, con $(i,j) \in Estaciones, m \in \{1,...,M\}$, de modo que

$ Y_{i,j,m} = \begin{cases} 1 \text{ si la línea } m \text{ para en el punto } (i,j)  & \\ 0 \text{ cc}
  \end{cases} $

$ Z_{i,j} = \begin{cases} 1 \text{ si alguna línea para en el punto } (i,j)  & \\ 0 \text{ cc}
  \end{cases} $


Las relaciones necearias son:

$X_{i,j,k,l,m} \leq Y_{i,j,m} ~~~ \forall ~(i,j,k,l) \in Conexiones\_validas, \forall ~m \in \{1,...,M\}$

$X_{i,j,k,l,m} \leq Y_{k,l,m} ~~~ \forall ~(i,j,k,l) \in Conexiones\_validas, \forall ~m \in \{1,...,M\}$

$ Y_{i,j,m} \leq ~~~ \sum_{(k,l) \in Estaciones: (i,j,k,l) \in Conexiones\_validas} X_{i,j,k,l,m} + \sum_{(k,l) \in Estaciones: (k,l,i,j,m) \in Conexiones\_validas} X_{k,l,i,j,m}\forall ~(i,j) \in Estaciones, \forall ~m \in \{1,...,M\}$

$Y_{i,j,m} \leq Z_{i,j} ~~~ \forall ~(i,j) \in Estaciones, \forall ~m \in \{1,...,M\} $

$ Z_{i,j} \leq ~~~ \sum_{m \in \{1,...,M\} } Y_{i,j,m}\forall ~(i,j) \in Estaciones$

Estas serán restricciones a considerar por el modelo también.

Finalmente, establecemos

$\sum_{(i,j) \in Estaciones} Z_{i,j}I_{i,j} \geq min\_estaciones\_especiales$



### **RESTRICCIÓN TIPO 3: número mínimo de distritos visitados** ###

Para esto vamos se definen nuevas variables binarias adicionales, $W_d, d=\{1,...,num\_distritos\}$:

$ W_{d} = \begin{cases} 1 \text{ si el distrito } d \text{ es visitado} & \\ 0 \text{ cc}
  \end{cases} $

Lo relacionaremos con el resto de variables mediante

$Z_{i,j} \leq W_{D_{i,j}}  ~~~\forall~ (i,j) \in Estaciones$

$ W_{d} \leq \sum_{(i,j) \in Estaciones : D(i,j) = d} Z_{i,j} ~~~\forall~ d \in\{1,...,num\_distritos\}$

Con esto, 

$ \sum_{d=1}^{num\_distritos}W_{d}  >= min\_distritos $


### **RESTRICCIÓN TIPO 4: Posición relativa entre estaciones** ###

Serán de utilidad nuevas matrices precalculadas

 $ H_{i,j,k,l} = \begin{cases} 1 \text{ si el punto } (i,j) \text{ está en la misma línea horizontal que } (k,l) & \\ 0 \text{ cc}
  \end{cases} $

$ V_{i,j,k,l} = \begin{cases} 1 \text{ si el punto } (i,j) \text{ está en la misma línea vertical que } (k,l) & \\ 0 \text{ cc}
  \end{cases} $

$ DIAG_{i,j,k,l} = \begin{cases} 1 \text{ si el punto } (i,j) \text{ está unido a } (k,l) \text{ por una línea diagonal} & \\ 0 \text{ cc}
  \end{cases} $

Así,

$H_{i,j,k,l} + V_{i,j,k,l} + DIAG_{i,j,k,l} \geq X_{i,j,k,l,m}~~~\forall ~~(i,j,k,l) \in Conexiones\_validas,~~ \forall~~ m \in \{1,...,M\}$



### **RESTRICCIÓN TIPO 5: estaciones intermedias** ###

Generamos la matriz

 $ INTER_{i,j,k,l} = \begin{cases} 1 \text{ si entre el punto } (i,j) \text{ y el  } (k,l) \text{ hay una estación} & \\ 0 \text{ cc}
  \end{cases} $

para finalmente establecer

$X_{i,j,k,l,m} \leq 1 - INTER_{i,j,k,l} ~~~\forall ~~(i,j,k,l) \in Conexiones\_validas,~~ \forall~~ m \in \{1,...,M\}$


### **RESTRICCIÓN TIPO 6: conexidad de las líneas** ###


Para las estaciones ráiz de cada línea, imponemos que sean parada obligatoria

  $ Y_{r_m,s_m,m} = 1 ~~ \forall ~m \in \{1,...,M\}, (r_m, s_m)$ raíz de $m$.

Y además, respecto al número de estaciones de cada línea, 

$\sum_{(i,j) \in Estaciones} Y_{i,j,m} = estaciones\_por\_linea ~~~\forall~ m \in \{1,...,M\}$

Pasamos a definir las variables de flujo

 $ f_{i,j,k,l,m} =$ cantidad de flujo que pasa por la arista entre las estaciones $(i,j)$ y $(k,l)$, para la línea $m$.

Será necesario

$ f_{i,j,k,l,m} \leq (estaciones\_por\_linea -1) X_{i,j,k,l,m} ~~~\forall ~~(i,j,k,l) \in Conexiones\_validas,~~ \forall~~ m \in \{1,...,M\}$

De esta forma, se definen las condiciones

* Para la raíz $(r_m,s_m)$ de la línea $m$:

  $  \sum_{(i,j)\in Estaciones : (r_m,s_m,i,j) \in Conexiones\_validas} f_{r_m,s_m,i,j,m}  - \sum_{(i,j)\in Estaciones : (i,j,r_m,s_m) \in Conexiones\_validas} f_{i,j,r_m,s_m,m} = - estaciones\_por\_linea +1,  ~~ \forall~~ m \in \{1,...,M\} $


* Para el resto de nodos $(i,j) \neq (r_m,s_m)$:

  $  \sum_{(k,l)\in Estaciones : (i,j,k,l) \in Conexiones_{validas}} f_{i,j,k,l,m}  - \sum_{(k,l)\in Estaciones : (k,l,i,j) \in Conexiones\_validas} f_{k,l,i,j} =  Y_{i,j,m}, ~~ \forall  ~(r_m,s_m)\neq (i,j) \in Estaciones ~~ ,\forall ~m \in \{1,...,M\} $





### **RESTRICCIÓN TIPO 7: dobles paradas** ###

Se definen 
 $ C_{i,j,m} =$ número de conexiones del punto $(i,j)$ en la línea $m$

Matemáticamente, 

$C_{i,j,m} = \sum_{(k,l) \in Estaciones : (i,j,k,l) \in Conexiones\_validas}X_{i,j,k,l,m} +  \sum_{(k,l) \in Estaciones : (k,l,i,j) \in Conexiones\_validas}X_{i,j,k,l,m} ~~~\forall ~~(i,j) \in Estaciones,~~ \forall~~ m \in \{1,...,M\}$


 Basta establecer entonces

  $ C_{i,j,m} \leq 2 ~~~\forall ~~(i,j) \in Estaciones,~~ \forall~~ m \in \{1,...,M\}$



### **RESTRICCIÓN TIPO 8: intersecciones entre vías** ###

Imponemos

$X_{i,j,k,l,m} + X_{p,q,r,s,t}  \leq 1 ~~~\forall ~~(i,j,k,l) \in Conexiones\_validas, (p,q,r,s) \in Segmentos\_secantes\_(i,j,k,l),~~ \forall~~ m,t \in \{1,...,M\}$



### **RESTRICCIÓN TIPO 9: conexión entre ambas mitades del río** ###

$\sum_{(i,j,k,l) \in Conexiones\_validas, m \in \{1,...,M\}} X_{i,j,k,l,m} |R_{i,j} - R_{k,l}| \geq 1 $


### **RESTRICCIÓN TIPO 10: paso por la estación central** ###

$Y_{c_1,c_2,m} \geq 1, ~~\forall~  m \in \{1,...,M\}$, con $(c_1,c_2) = estacion\_central  $




## **RESUMEN DE RESTRICCIONES** ##

Reescribimos las restricciones ahora reagrupando términos en la forma estándar:


1. $\sum_{(i,j,k,l) \in Conexiones\_validas,m \in \{1,...,M\} }  DIST_{i,j,k,l,}X_{i,j,k,l,m} \leq max\_long$



1.   $X_{i,j,k,l,m} - Y_{i,j,m} \leq 0 ~~~ \forall ~(i,j,k,l) \in Conexiones\_validas, \forall ~m \in \{1,...,M\}$

     $X_{i,j,k,l,m} - Y_{k,l,m} \leq 0 ~~~ \forall ~(i,j,k,l) \in Conexiones\_validas, \forall ~m \in \{1,...,M\}$

3.   $ Y_{i,j,m} - \sum_{(k,l) \in Estaciones: (i,j,k,l) \in Conexiones\_validas}  X_{i,j,k,l,m} - \sum_{(k,l) \in Estaciones: (k,l,i,j) \in Conexiones\_validas}  X_{k,l,i,j,m}\leq 0 ~~~ \forall ~(i,j) \in Estaciones, \forall ~m \in \{1,...,M\}$


4. $Y_{i,j,m} - Z_{i,j} \leq 0  ~~~ \forall ~(i,j) \in Estaciones,  \forall ~m \in \{1,...,M\} $


5. $ Z_{i,j} - \sum_{m \in \{1,...,M\} }  Y_{i,j,m} \leq 0 ~~~ \forall ~(i,j) \in Estaciones$

6. $- \sum_{(i,j) \in Estaciones} Z_{i,j}I_{i,j} \leq - min\_estaciones\_especiales$

7. $Z_{i,j} - W_{D_{i,j}} \leq 0 ~~~\forall~ (i,j) \in Estaciones$

8. $ W_{d} -\sum_{(i,j) \in Estaciones : D(i,j) = d} Z_{i,j}\leq 0 ~~~\forall~ d \in\{1,...,num\_distritos\}$

9. $- \sum_{d=1}^{num\_distritos}W_{d}  \leq - min\_distritos $

10. $X_{i,j,k,l,m} \leq H_{i,j,k,l} + V_{i,j,k,l} + DIAG_{i,j,k,l} ~~~\forall ~~(i,j,k,l) \in Conexiones\_validas,~~ \forall~~ m \in \{1,...,M\}$

11. $X_{i,j,k,l,m} \leq 1 - INTER_{i,j,k,l} ~~~\forall ~~(i,j,k,l) \in Conexiones\_validas,~~ \forall~~ m \in \{1,...,M\}$

12. $ Y_{r_m,s_m,m} = 1, ~~ \forall ~m \in \{1,...,M\}, (r_m, s_m)$ raíz de $m$.

13. $ \sum_{(i,j) \in Estaciones} Y_{i,j,m}  = estaciones\_por\_linea ~~~\forall~ m \in \{1,...,M\}$


14. $ f_{i,j,k,l,m}  + (1-estaciones\_por\_linea) X_{i,j,k,l,m} \leq 0 ~~~\forall ~~(i,j,k,l) \in Conexiones\_validas,~~ \forall~~ m \in \{1,...,M\}$

15.  $  \sum_{(i,j)\in Estaciones : (r_m,s_m,i,j) \in Conexiones\_validas} f_{r_m,s_m,i,j,m}  - \sum_{(i,j)\in Estaciones : (i,j,r_m,s_m) \in Conexiones\_validas} f_{i,j,r_m,s_m,m} = - estaciones\_por\_linea +1,  ~~ \forall~~ m \in \{1,...,M\} $


16.  $  \sum_{(k,l)\in Estaciones : (i,j,k,l) \in Conexiones_{validas}} f_{i,j,k,l,m}  - \sum_{(k,l)\in Estaciones : (k,l,i,j) \in Conexiones\_validas} f_{k,l,i,j}  - Y_{i,j,m}=0, ~~ \forall  ~(r_m,s_m)\neq (i,j) \in Estaciones ~~ ,\forall ~m \in \{1,...,M\} $


17. $C_{i,j,m} - \sum_{(k,l) \in Estaciones : (i,j,k,l) \in Conexiones\_validas}X_{i,j,k,l,m}  -  \sum_{(k,l) \in Estaciones : (k,l,i,j) \in Conexiones\_validas}X_{k,l,i,j,m}= 0 ~~~\forall ~~(i,j) \in Estaciones,~~ \forall~~ m \in \{1,...,M\}$

18.  $ C_{i,j,m} \leq 2 ~~~\forall ~~(i,j) \in Estaciones,~~ \forall~~ m \in \{1,...,M\}$


19. $X_{i,j,k,l,m} + X_{p,q,r,s,t}  \leq 1 ~~~\forall ~~(i,j,k,l) \in Conexiones\_validas, (p,q,r,s) \in Segmentos\_secantes\_(i,j,k,l),~~ \forall~~ m,t \in \{1,...,M\}$

20. $ - \sum_{(i,j,k,l) \in Conexiones\_validas, m \in \{1,...,M\}} X_{i,j,k,l,m} |R_{i,j} - R_{k,l}| \leq -1 $


21. $Y_{c_1,c_2,m}  = 1, ~~\forall~  m \in \{1,...,M\}, (c_1,c_2) = estacion\_central$


# **ESCRITURA COMO PROBLEMA DE PL** 

Tenemos las variables $X_{i,j,k,l,m}, Y_{i,j,m}, Z_{i,j}, W_d, C_{i,j,m},  f_{i,j,k,l,m}$. Agrupándolas todas, para formular un problema de PL del tipo $Ax \leq b$,

Para esto en necesario definir los índices que cada una tendrá en el vector $x$.

In [ ]:

def generar_mapeo_indices():

    """
    Genera un mapeo de índices para las variables de decisión en el problema.
    Devuelve un diccionario que almacena los índices asociados a cada variable de decisión
    en el vector plano, así como el tamaño total del vector.
    """

    # Diccionario que almacenará los índices
    indice_en_x = {}

    # Índice inicial en el vector plano
    contador = 0


    # Mapear X_{i,j,k,l,m}
    for i,j,k,l in conexiones_validas:
        for m in range(M):
             indice_en_x[f"X_{i}_{j}_{k}_{l}_{m}"] = contador
             contador += 1


    # Mapear Y_{i,j,m}
    for i, j in estaciones:
          for m in range(M):
              indice_en_x[f"Y_{i}_{j}_{m}"] = contador
              contador += 1


    # Mapear Z_{i,j}
    for i,j in estaciones:
          indice_en_x[f"Z_{i}_{j}"] = contador
          contador += 1


    # Mapear W_d
    for d in range(1, num_distritos+1):
        indice_en_x[f"W_{d}"] = contador
        contador += 1

    # Mapear C_{i,j,m}
    for i, j in estaciones:
        for m in range(M):
            indice_en_x[f"C_{i}_{j}_{m}"] = contador
            contador += 1

    # Mapear f_{i,j,k,l,m}
    for i, j, k, l in conexiones_validas:
          for m in range(M):
              indice_en_x[f"f_{i}_{j}_{k}_{l}_{m}"] = contador
              contador += 1

    return indice_en_x, contador

In [ ]:
indice_en_x, n_cols = generar_mapeo_indices()

print("Número de variables: ", n_cols)

1.  $\sum_{(i,j,k,l) \in Conexiones\_validas,m \in \{1,...,M\} }  DIST_{i,j,k,l,}X_{i,j,k,l,m} \leq max\_long$


In [ ]:
def r1():
    """
    Devuelve las filas del sistema correspondientes a la restricción 1.
    """

    rows =  lil_matrix((1, n_cols))
    b = np.zeros(1)

    for i,j,k,l in conexiones_validas:
        for m in range(M):
            
            rows[0, indice_en_x[f"X_{i}_{j}_{k}_{l}_{m}"]] = DIST[i,j,k,l]
    
    b[0] = max_long

    return rows, b

2.   $X_{i,j,k,l,m} - Y_{i,j,m} \leq 0 ~~~ \forall ~(i,j,k,l) \in Conexiones\_validas, \forall ~m \in \{1,...,M\}$

     $X_{i,j,k,l,m} - Y_{k,l,m} \leq 0 ~~~ \forall ~(i,j,k,l) \in Conexiones\_validas, \forall ~m \in \{1,...,M\}$

In [ ]:
def r2():
    """
    Devuelve las filas del sistema correspondientes a la restricción 2.
    """

    n_rows = 2 * N_val * M  # Número total de este tipo de restricciones
    rows = lil_matrix((n_rows, n_cols))
    b = np.zeros(n_rows)  

    row_idx = 0

    for i,j,k,l in conexiones_validas:
        for m in range(M):

            rows[row_idx, indice_en_x[f"X_{i}_{j}_{k}_{l}_{m}"]] = 1
            rows[row_idx, indice_en_x[f"Y_{i}_{j}_{m}"]] = -1
            b[row_idx] = 0

            rows[N_val*M + row_idx, indice_en_x[f"X_{i}_{j}_{k}_{l}_{m}"]] = 1
            rows[N_val*M + row_idx, indice_en_x[f"Y_{k}_{l}_{m}"]] = -1
            b[N_val*M + row_idx ] = 0

            row_idx += 1

    return rows, b

3.    $ Y_{i,j,m} - \sum_{(k,l) \in Estaciones: (i,j,k,l) \in Conexiones\_validas}  X_{i,j,k,l,m} - \sum_{(k,l) \in Estaciones: (k,l,i,j) \in Conexiones\_validas}  X_{k,l,i,j,m}\leq 0 ~~~ \forall ~(i,j) \in Estaciones, \forall ~m \in \{1,...,M\}$

In [ ]:
def r3():
    """
    Devuelve las filas del sistema correspondientes a la restricción 3.
    """

    n_rows = n_estaciones * M
    rows =  lil_matrix((n_rows, n_cols))
    b = np.zeros(n_rows)

    row_idx = 0

    for i,j in estaciones:
          for m in range(M):

              rows[row_idx, indice_en_x[f"Y_{i}_{j}_{m}"]] = 1

              for k,l in estaciones:

                      if (i,j,k,l) in conexiones_validas:
                        rows[row_idx, indice_en_x[f"X_{i}_{j}_{k}_{l}_{m}"]] = -1

                      if (k,l,i,j) in conexiones_validas:
                        rows[row_idx, indice_en_x[f"X_{k}_{l}_{i}_{j}_{m}"]] = -1

              b[row_idx] = 0
              row_idx += 1

    return rows, b

4. $Y_{i,j,m} - Z_{i,j} \leq 0  ~~~ \forall ~(i,j) \in Estaciones,  \forall ~m \in \{1,...,M\} $




In [ ]:
def r4():
    """
    Devuelve las filas del sistema correspondientes a la restricción 4.
    """

    n_rows = n_estaciones * M
    rows =  lil_matrix((n_rows, n_cols))
    b = np.zeros(n_rows)

    row_idx = 0

    for i,j in estaciones:
          for m in range(M):

            rows[row_idx, indice_en_x[f"Y_{i}_{j}_{m}"]] = 1
            rows[row_idx, indice_en_x[f"Z_{i}_{j}"]] = -1

            b[row_idx] = 0
            row_idx += 1

    return rows, b


5. $ Z_{i,j} - \sum_{m \in \{1,...,M\} }  Y_{i,j,m} \leq 0 ~~~ \forall ~(i,j) \in Estaciones$


In [ ]:
def r5():
    """
    Devuelve las filas del sistema correspondientes a la restricción 5.
    """

    n_rows = n_estaciones
    rows =  lil_matrix((n_rows, n_cols))
    b = np.zeros(n_rows)

    row_idx = 0

    for i,j in estaciones:

        rows[row_idx, indice_en_x[f"Z_{i}_{j}"]] = 1

        for m in range(M):
            rows[row_idx, indice_en_x[f"Y_{i}_{j}_{m}"]] = -1


        b[row_idx] = 0
        row_idx += 1


    return rows, b

6. $- \sum_{(i,j) \in Estaciones} Z_{i,j}I_{i,j} \leq - min\_estaciones\_especiales$

In [ ]:
def r6():
    """
    Devuelve las filas del sistema correspondientes a la restricción 6.
    """

    rows =  lil_matrix((1, n_cols))
    b = np.zeros(1)

    for i,j in estaciones:
            rows[0, indice_en_x[f"Z_{i}_{j}"]] = -I[i,j]

    b[0] = -min_estaciones_especiales

    return rows, b

7. $Z_{i,j} - W_{D_{i,j}} \leq 0 ~~~\forall~ (i,j) \in Estaciones$


In [ ]:
def r7():
    """
    Devuelve las filas del sistema correspondientes a la restricción 7.
    """

    n_rows = n_estaciones
    rows =  lil_matrix((n_rows, n_cols))
    b = np.zeros(n_rows)

    row_idx = 0

    for i,j in estaciones:
            
            rows[row_idx, indice_en_x[f"Z_{i}_{j}"]] = 1
            dij = D[i,j]
            rows[row_idx, indice_en_x[f"W_{dij}"]] = -1

            b[row_idx] = 0
            row_idx += 1

    return rows, b

8. $ W_{d} -\sum_{(i,j) \in Estaciones : D(i,j) = d} Z_{i,j}\leq 0 ~~~\forall~ d \in\{1,...,num\_distritos\}$

In [ ]:
def r8():
    """
    Devuelve las filas del sistema correspondientes a la restricción 8.
    """

    n_rows = num_distritos
    rows =  lil_matrix((n_rows, n_cols))
    b = np.zeros(n_rows)

    row_idx = 0

    for d in range(1, num_distritos+1):

        rows[row_idx, indice_en_x[f"W_{d}"]] = 1

        for i,j in estaciones:

            if D[i,j] == d:
                rows[row_idx, indice_en_x[f"Z_{i}_{j}"]] = -1

        b[row_idx] = 0
        row_idx += 1

    return rows, b

9. $- \sum_{d=1}^{num\_distritos}W_{d}  \leq - min\_distritos $


In [ ]:
def r9():
    """
    Devuelve las filas del sistema correspondientes a la restricción 9.
    """

    rows =  lil_matrix((1, n_cols))
    b = np.zeros(1)

    for d in range(1, num_distritos+1):
          
          rows[0, indice_en_x[f"W_{d}"]] = -1
          
    b[0] = -min_distritos

    return rows, b


10. $X_{i,j,k,l,m} \leq H_{i,j,k,l} + V_{i,j,k,l} + DIAG_{i,j,k,l} ~~~\forall ~~(i,j,k,l) \in Conexiones\_validas,~~ \forall~~ m \in \{1,...,M\}$

In [ ]:
def r10():
    """
    Devuelve las filas del sistema correspondientes a la restricción 10.
    """
    
    n_rows = N_val * M
    rows =  lil_matrix((n_rows, n_cols))
    b = np.zeros(n_rows)

    row_idx = 0

    for i,j,k,l in conexiones_validas:
          for m in range(M):

              rows[row_idx, indice_en_x[f"X_{i}_{j}_{k}_{l}_{m}"]] = 1

              b[row_idx] = H[i,j,k,l] + V[i,j,k,l] + DIAG[i,j,k,l]

              row_idx += 1

    return rows, b

11. $X_{i,j,k,l,m} \leq 1 - INTER_{i,j,k,l} ~~~\forall ~~(i,j,k,l) \in Conexiones\_validas,~~ \forall~~ m \in \{1,...,M\}$

In [ ]:
def r11():
    """
    Devuelve las filas del sistema correspondientes a la restricción 11.
    """

    n_rows = N_val * M
    rows = lil_matrix((n_rows, n_cols))
    b = np.zeros(n_rows)

    row_idx = 0

    for i,j,k,l in conexiones_validas:
          for m in range(M):

              rows[row_idx, indice_en_x[f"X_{i}_{j}_{k}_{l}_{m}"]] = 1

              b[row_idx] = 1 - INTER[i,j,k,l]

              row_idx += 1

    return rows, b

12. $ Y_{r_m,s_m,m} = 1, ~~ \forall ~m \in \{1,...,M\}, (r_m, s_m)$ raíz de $m$.

In [ ]:
def r12():
    """
    Devuelve las filas del sistema correspondientes a la restricción 12.
    """

    n_rows = M
    rows = lil_matrix((n_rows, n_cols))
    b = np.zeros(n_rows)

    row_idx = 0

    for (r,s) in raices:
        
        rows[row_idx, indice_en_x[f"Y_{r}_{s}_{row_idx}"]] = 1
        b[row_idx] = 1
        row_idx += 1

    return rows, b


13. $\sum_{(i,j) \in Estaciones} Y_{i,j,m} =  estaciones\_por\_linea ~~~\forall~ m \in \{1,...,M\}$

In [ ]:
def r13():
    """
    Devuelve las filas del sistema correspondientes a la restricción 13.
    """

    n_rows = N_val * M
    rows = lil_matrix((n_rows, n_cols))
    b = np.zeros(n_rows)

    row_idx = 0

    for i,j,k,l in conexiones_validas:
        for m in range(M):
            
            rows[row_idx, indice_en_x[f"X_{i}_{j}_{k}_{l}_{m}"]] = 1-estaciones_por_linea
            rows[row_idx, indice_en_x[f"f_{i}_{j}_{k}_{l}_{m}"]] = 1
            b[row_idx] = 0
            row_idx += 1

    return rows, b

14. $ f_{i,j,k,l,m}  + (1-estaciones\_por\_linea) X_{i,j,k,l,m} \leq 0 ~~~\forall ~~(i,j,k,l) \in Conexiones\_validas,~~ \forall~~ m \in \{1,...,M\}$

In [ ]:
def r14():
    """
    Devuelve las filas del sistema correspondientes a la restricción 14.
    """

    n_rows = M
    rows =  lil_matrix((n_rows, n_cols))
    b = np.zeros(n_rows)

    for row_idx in range(n_rows): #row_idx = m
        for i,j in estaciones:
                
                rows[row_idx, indice_en_x[f"Y_{i}_{j}_{row_idx}"]] = 1

        b[row_idx] = estaciones_por_linea

    return rows, b

15.  $  \sum_{(i,j)\in Estaciones : (r_m,s_m,i,j) \in Conexiones\_validas} f_{r_m,s_m,i,j,m}  - \sum_{(i,j)\in Estaciones : (i,j,r_m,s_m) \in Conexiones\_validas} f_{i,j,r_m,s_m,m} = - estaciones\_por\_linea +1,  ~~ \forall~~ m \in \{1,...,M\} $

In [ ]:
def r15():
    """
    Devuelve las filas del sistema correspondientes a la restricción 15.
    """

    n_rows = M
    rows = lil_matrix((n_rows, n_cols))
    b = np.zeros(n_rows)

    for m in range(M):

        (r,s) = raices[m]
        
        for i,j in estaciones:

            if (r,s,i,j) in conexiones_validas:
                rows[m, indice_en_x[f"f_{r}_{s}_{i}_{j}_{m}"]] = 1

            if (i,j,r,s) in conexiones_validas:
                rows[m, indice_en_x[f"f_{i}_{j}_{r}_{s}_{m}"]] = -1

        b[m] = 1-estaciones_por_linea

    return rows, b

16.  $  \sum_{(k,l)\in Estaciones : (i,j,k,l) \in Conexiones_{validas}} f_{i,j,k,l,m}  - \sum_{(k,l)\in Estaciones : (k,l,i,j) \in Conexiones\_validas} f_{k,l,i,j}  - Y_{i,j,m}=0, ~~ \forall  ~(r_m,s_m)\neq (i,j) \in Estaciones ~~ ,\forall ~m \in \{1,...,M\} $

In [ ]:
def r16():
    """
    Devuelve las filas del sistema correspondientes a la restricción 16.
    """

    n_rows = (n_estaciones-1) * M
    rows = lil_matrix((n_rows, n_cols))
    b = np.zeros(n_rows)

    row_idx = 0

    for m in range(M):

        (r,s) = raices[m]

        for i,j in estaciones:

            if (i,j)!=(r,s):

                rows[row_idx, indice_en_x[f"Y_{i}_{j}_{m}"]] = -1
                b[row_idx] = 0

                for k,l in estaciones:

                        if (i,j,k,l) in conexiones_validas:
                            rows[row_idx, indice_en_x[f"f_{i}_{j}_{k}_{l}_{m}"]] = 1

                        if (k,l,i,j) in conexiones_validas:
                            rows[row_idx, indice_en_x[f"f_{k}_{l}_{i}_{j}_{m}"]] = -1

                row_idx += 1


    return rows, b

17. $C_{i,j,m} - \sum_{(k,l) \in Estaciones : (i,j,k,l) \in Conexiones\_validas}X_{i,j,k,l,m}  -  \sum_{(k,l) \in Estaciones : (k,l,i,j) \in Conexiones\_validas}X_{k,l,i,j,m}= 0 ~~~\forall ~~(i,j) \in Estaciones,~~ \forall~~ m \in \{1,...,M\}$

In [ ]:
def r17():
    """
    Devuelve las filas del sistema correspondientes a la restricción 17.
    """

    n_rows = n_estaciones * M
    rows = lil_matrix((n_rows, n_cols))
    b = np.zeros(n_rows)

    row_idx = 0

    for i,j in estaciones:
              for m in range(M):

                  rows[row_idx, indice_en_x[f"C_{i}_{j}_{m}"]] = 1

                  for k,l in estaciones:

                          if (i,j,k,l) in conexiones_validas:
                            rows[row_idx, indice_en_x[f"X_{i}_{j}_{k}_{l}_{m}"]] = -1

                          if (k,l,i,j) in conexiones_validas:
                            rows[row_idx, indice_en_x[f"X_{k}_{l}_{i}_{j}_{m}"]] = -1

                  b[row_idx] = 0
                  row_idx += 1

    return rows, b

18.  $ C_{i,j,m} \leq 2 ~~~\forall ~~(i,j) \in Estaciones,~~ \forall~~ m \in \{1,...,M\}$


In [ ]:
def r18():
    """
    Devuelve las filas del sistema correspondientes a la restricción 18.
    """

    n_rows = n_estaciones * M
    rows =  lil_matrix((n_rows, n_cols))
    b = np.zeros(n_rows)

    row_idx = 0

    for i,j in estaciones:
              for m in range(M):

                  rows[row_idx, indice_en_x[f"C_{i}_{j}_{m}"]] = 1

                  b[row_idx] = 2
                  row_idx += 1

    return rows, b


19. $X_{i,j,k,l,m} + X_{p,q,r,s,t}  \leq 1 ~~~\forall ~~(i,j,k,l) \in Conexiones\_validas, (p,q,r,s) \in Segmentos\_secantes\_(i,j,k,l),~~ \forall~~ m,t \in \{1,...,M\}$

In [ ]:
def r19():
    """
    Devuelve las filas del sistema correspondientes a la restricción 19.
    """

    row_list  = []
    b_list = []

    row_idx = 0

    for i,j,k,l in conexiones_validas:

        segs_secantes = segmentos_secantes(i,j,k,l)

        for p,q,r,s in segs_secantes:
            for m in range(M):
                for t in range(M):

                    row_list.append((row_idx, f"X_{i}_{j}_{k}_{l}_{m}", 1))
                    row_list.append((row_idx, f"X_{p}_{q}_{r}_{s}_{t}", 1))
                    b_list.append((row_idx, 1))
                    row_idx += 1

    rows = lil_matrix((row_idx, n_cols))
    b = np.zeros(row_idx)
    for idx, var_name, value in row_list:
        rows[idx, indice_en_x[var_name]] = value
    for idx, value in b_list:
        b[idx] = value

    return rows, b

20. $ - \sum_{(i,j,k,l) \in Conexiones\_validas, m \in \{1,...,M\}} X_{i,j,k,l,m} |R_{i,j} - R_{k,l}| \leq -1 $


In [ ]:
def r20():
    """
    Devuelve las filas del sistema correspondientes a la restricción 20.
    """

    rows =  lil_matrix((1, n_cols))
    b = np.zeros(1)

    for m in range(M):
        for i,j,k,l in conexiones_validas:

            coef = -1 * abs(R[i,j] - R[k,l])
            rows[0, indice_en_x[f"X_{i}_{j}_{k}_{l}_{m}"]] = coef

    b[0] = -1
    
    return rows, b


21. $Y_{c_1,c_2,m} = 1, ~~\forall~  m \in \{1,...,M\}$, con $(c_1,c_2) = estacion\_central $


In [ ]:
def r21():
    """
    Devuelve las filas del sistema correspondientes a la restricción 21.
    """

    n_rows = M

    rows =  lil_matrix((n_rows, n_cols))
    b = np.zeros(n_rows)

    (c1,c2) = estacion_central

    for m in range(M):

        rows[m, indice_en_x[f"Y_{c1}_{c2}_{m}"]] = 1
        b[m] = 1

    return rows, b

# **SOLVER** #

In [ ]:
# Generación de las restricciones (filas del sistema).
rows1, b1 = r1()
rows2, b2 = r2()
rows3, b3 = r3()
rows4, b4 = r4()
rows5, b5 = r5()
rows6, b6 = r6()
rows7, b7 = r7()
rows8, b8 = r8()
rows9, b9 = r9()
rows10, b10 = r10()
rows11, b11 = r11()
rows12, b12 = r12()
rows13, b13 = r13()
rows14, b14 = r14()
rows15, b15 = r15()
rows16, b16 = r16()
rows17, b17 = r17()
rows18, b18 = r18()
rows19, b19 = r19()
rows20, b20 = r20()
rows21, b21 = r21()

In [ ]:
# Agrupación de las restricciones de igualdad
A_eq = vstack([rows12, rows13, rows15, rows16, rows17, rows21], format='csr')
b_eq = np.hstack([b12, b13, b15, b16, b17, b21])

# Agrupación de las restricciones de desigualdad
A_leq = vstack([rows1, rows2, rows3, rows4, rows5, rows6, rows7, rows8, rows9, rows10, rows11, rows14, rows18, rows19, rows20], format='csr')
b_leq = np.hstack([b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b14, b18, b19, b20])

# Impresión de las dimensiones de las matrices, 
# representando (número de restricciones, número de variables)
print("A_eq shape", A_eq.shape)
print("b_eq shape", np.shape(b_eq))
print("A_leq shape", A_leq.shape)
print("b_leq shape", np.shape(b_leq))


### MINIMIZACIÓN DE LONGITUD DE VÍAS ####

In [ ]:
# === 1. Crear un vector o lista plana de longitud el número de variables ===
x = [None] * n_cols

# === 2. Crear las variables de decisión para el solver, asociándolas al vector plano ===
for nombre, idx in indice_en_x.items(): # nombre = X_{i}_{j}_{k}_{l}_{m}, idx = indices_en_x[nombre]
    if nombre.startswith("X_"):
        x[idx] = pulp.LpVariable(nombre, cat='Binary')
    elif nombre.startswith("Y_"):
        x[idx] = pulp.LpVariable(nombre, cat='Binary')
    elif nombre.startswith("Z_"):
        x[idx] = pulp.LpVariable(nombre, cat='Binary')
    elif nombre.startswith("C_"):
        x[idx] = pulp.LpVariable(nombre, lowBound=0, cat='Integer')
    elif nombre.startswith("f_"):
        x[idx] = pulp.LpVariable(nombre, lowBound=0, cat='Integer')
    elif nombre.startswith("W_"):
        x[idx] = pulp.LpVariable(nombre, cat='Binary')
    elif nombre.startswith("AUX_"):
        x[idx] = pulp.LpVariable(nombre, cat='Binary')
    elif nombre.startswith("L_"):
        x[idx] = pulp.LpVariable(nombre, cat= 'Binary')


# === 3. Crear el modelo ===
prob = pulp.LpProblem("Optimización_MILP", pulp.LpMinimize)

# === 4. Definir la función objetivo ===
prob += pulp.lpSum(DIST[i, j, k, l] * x[indice_en_x[f"X_{i}_{j}_{k}_{l}_{m}"]] for i, j, k, l in conexiones_validas for m in range(M))

# === 5. Especificar las restricciones ===
for idx in range(A_leq.shape[0]):
    fila = A_leq.getrow(idx)
    prob += pulp.lpSum(fila.data[k] * x[int(fila.indices[k])] for k in range(len(fila.indices))) <= b_leq[idx]

for idx in range(A_eq.shape[0]):
    fila = A_eq.getrow(idx)
    prob += pulp.lpSum(fila.data[k] * x[int(fila.indices[k])] for k in range(len(fila.indices))) == b_eq[idx]

# === 6. Resolver ===
prob.solve()

# === 7. Obtener resultados ===
if pulp.LpStatus[prob.status] == 'Optimal':
    print("Solución óptima encontrada:")
    for var in prob.variables():
        if var.varValue > 0:
            print(f"{var.name} = {var.varValue}")  
    print("\n Valor objetivo:", pulp.value(prob.objective))
else:
    print("No se encontró solución:", pulp.LpStatus[prob.status])

In [ ]:
def graficar_escenario(X=None):
    """
    Función para representar la ciudad, sus estaciones y conexiones.
    X: matriz de variables de decisión (opcional)
    """

    fig, ax = plt.subplots(figsize=(8, 8))

    # Crear una cuadrícula de 10x10
    ax.set_xticks(np.arange(0, N, 1))
    ax.set_yticks(np.arange(0, N, 1))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.grid(True, which='both', color = "lightgray")

    for i in range(N):
        for j in range(N):
            if E[i, j] == 1:
                if I[i, j] == 1:  # Si es una estación especial (estrella)
                    if R[i, j] == 0:
                        ax.plot(j, N-i-1, '*', color="#00B4A0", markersize=12, markeredgewidth=2)
                    else:
                      ax.plot(j, N-i-1, '*', color="#0000DC", markersize=12, markeredgewidth=2)
                else:  # Si es una estación normal
                    if (i,j) == estacion_central:
                            markeredgewidth = 2.5
                            type = 'D'
                    else:
                            markeredgewidth = 1
                            type = 'o'
                    # Verificar la mitad del río usando la matriz R
                    if R[i, j] == 0:
                        ax.plot(j, N-i-1, type, color="#00B4A0", markersize = 10, markerfacecolor='none', markeredgewidth=markeredgewidth)
                    else:
                        ax.plot(j, N-i-1, type, color= "#0000DC", markersize = 10, markerfacecolor='none')
            else:  # Si no es una estación
                ax.plot(j, N-i-1, 'o', color='gray', markersize=3)

    # Líneas de separación entre distritos
    for i in [3,7]:
        ax.axhline(y=i-0.5, color='#FFDD44', linestyle = '--', linewidth=1)
        ax.axvline(x=i-0.5, color='#FFDD44',linestyle = '--',  linewidth=1)

    ax.plot([0.5, 0.5],[-0.5,0.5], color='#FFDD44', linestyle='--', linewidth=1)
    ax.plot([-0.5,0.5],[0.5, 0.5], color='#FFDD44', linestyle='--', linewidth=1)

    ax.plot([9.5, 8.5],[0.5,0.5], color='#FFDD44', linestyle='--', linewidth=1)
    ax.plot([8.5,8.5],[-0.5, 0.5], color='#FFDD44', linestyle='--', linewidth=1)

    ax.plot([0.5, 0.5],[8.5,9.5], color='#FFDD44', linestyle='--', linewidth=1)
    ax.plot([-0.5,0.5],[8.5, 8.5], color='#FFDD44', linestyle='--', linewidth=1)

    ax.plot([9.5, 8.5],[8.5,8.5], color='#FFDD44', linestyle='--', linewidth=1)
    ax.plot([8.5, 8.5],[8.5,9.5], color='#FFDD44', linestyle='--', linewidth=1)

    if(X is not None):
      colors = [plt.cm.tab10(i) for i in range(1,M+1)]
      for m in range(M):
          for i in range (N):
            for j in range (N):
              for k in range (N):
                for l in range (N):
                  if X[i,j,k,l,m]==1:
                      ax.plot([j,l], [N-i-1,N-k-1], color=colors[m], linewidth=2)

    # Ajustar límites
    ax.set_xlim(-0.5, N-0.5)
    ax.set_ylim(-0.5, N-0.5)

    # Mostrar el gráfico
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()
    plt.close(fig)

In [ ]:
# Graficar escenario base
graficar_escenario(X=None)

In [ ]:
# Generar matriz final de variables de tipo X para graficar el resultado

matriz_resultado = np.zeros((N, N, N, N, M), dtype=int)
for var in prob.variables():
    if var.name.startswith("X_") and var.varValue == 1:
        match = re.match(r"X_(\d+)_(\d+)_(\d+)_(\d+)_(\d+)", var.name)
        if match:
            i, j, k, l, m = map(int, match.groups())
            matriz_resultado[i, j, k, l, m] = 1

graficar_escenario(X=matriz_resultado)

In [ ]:
# Imprimir el número de estaciones importantes visitadas
n_est_imp_visitadas_min = sum([I[i,j] * x[indice_en_x[f"Z_{i}_{j}"]].varValue for i,j in estaciones])
print("Número de estaciones importantes visitadas: ", n_est_imp_visitadas_min)

# Imprimir la distancia total recorrida
dist_total_min = sum([DIST[i,j,k,l] * x[indice_en_x[f"X_{i}_{j}_{k}_{l}_{m}"]].varValue for i,j,k,l in conexiones_validas for m in range(M)])
print("Distancia mínima total recorrida: ", dist_total_min)

# Se puede evitar este último cálculo y usar el valor de la función objetivo: 
# dist_total = pulp.value(prob.objective). Efectivamente, 
print(dist_total_min == pulp.value(prob.objective))




### MAXIMIZACIÓN DE ESTACIONES ESPECIALES VISITADAS ###

In [ ]:
# === 3. Crear el modelo ===
prob = pulp.LpProblem("Optimización_MILP", pulp.LpMinimize)

# === 4. Definir la función objetivo ===
prob += - pulp.lpSum( I[i,j] * x[indice_en_x[f"Z_{i}_{j}"]] for i, j in estaciones)

# === 5. Especificar las restricciones ===
for idx in range(A_leq.shape[0]):
    fila = A_leq.getrow(idx)
    prob += pulp.lpSum(fila.data[k] * x[int(fila.indices[k])] for k in range(len(fila.indices))) <= b_leq[idx]

for idx in range(A_eq.shape[0]):
    fila = A_eq.getrow(idx)
    prob += pulp.lpSum(fila.data[k] * x[int(fila.indices[k])] for k in range(len(fila.indices))) == b_eq[idx]

# === 6. Resolver ===
prob.solve()

# === 7. Obtener resultados ===
if pulp.LpStatus[prob.status] == 'Optimal':
    print("Solución óptima encontrada:")
    for var in prob.variables():
        if var.varValue > 0:
            print(f"{var.name} = {var.varValue}")  
    print("\n Valor objetivo:", pulp.value(prob.objective))
else:
    print("No se encontró solución:", pulp.LpStatus[prob.status])


In [ ]:
matriz_resultado = np.zeros((N, N, N, N, M), dtype=int)
for var in prob.variables():
    if var.name.startswith("X_") and var.varValue == 1:
        match = re.match(r"X_(\d+)_(\d+)_(\d+)_(\d+)_(\d+)", var.name)
        if match:
            i, j, k, l, m = map(int, match.groups())
            matriz_resultado[i, j, k, l, m] = 1

graficar_escenario(X=matriz_resultado)

In [ ]:
# Imprimir el número de estaciones importantes visitadas
n_est_imp_visitadas_max = sum([I[i,j] * x[indice_en_x[f"Z_{i}_{j}"]].varValue for i,j in estaciones])
print("Número de estaciones importantes visitadas: ", n_est_imp_visitadas_max)
# También se puede evitar cálculo y usar el valor de la función objetivo: 
# n_est_imp_visitadas = - pulp.value(prob.objective)

# Imprimir la distancia total recorrida
dist_total_max = sum([DIST[i,j,k,l] * x[indice_en_x[f"X_{i}_{j}_{k}_{l}_{m}"]].varValue for i,j,k,l in conexiones_validas for m in range(M)])
print("Distancia mínima total recorrida: ", dist_total_max)

#### MINIMIZACIÓN DE LONGITUD DE VÍAS VISITANDO EL MÁXIMO NÚMERO DE ESTACIONES ESPECIALES ###

In [ ]:
min_estaciones_especiales = 9

rows6, b6 = r6()

A_leq = vstack([rows1, rows2, rows3, rows4, rows5, rows6, rows7, rows8, rows9, rows10, rows11, rows14, rows18, rows19, rows20], format='csr')
b_leq = np.hstack([b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b14, b18, b19, b20])

In [ ]:
# === 3. Crear el modelo ===
prob = pulp.LpProblem("Optimización_MILP", pulp.LpMinimize)

# === 4. Definir la función objetivo ===
prob += pulp.lpSum(DIST[i, j, k, l] * x[indice_en_x[f"X_{i}_{j}_{k}_{l}_{m}"]] for i, j, k, l in conexiones_validas for m in range(M))

# === 5. Especificar las restricciones ===
for idx in range(A_leq.shape[0]):
    fila = A_leq.getrow(idx)
    prob += pulp.lpSum(fila.data[k] * x[int(fila.indices[k])] for k in range(len(fila.indices))) <= b_leq[idx]

for idx in range(A_eq.shape[0]):
    fila = A_eq.getrow(idx)
    prob += pulp.lpSum(fila.data[k] * x[int(fila.indices[k])] for k in range(len(fila.indices))) == b_eq[idx]

# === 6. Resolver ===
prob.solve()

# === 7. Obtener resultados ===
if pulp.LpStatus[prob.status] == 'Optimal':
    print("Solución óptima encontrada:")
    for var in prob.variables():
        if var.varValue > 0:
            print(f"{var.name} = {var.varValue}")  
    print("\n Valor objetivo:", pulp.value(prob.objective))
else:
    print("No se encontró solución:", pulp.LpStatus[prob.status])

In [ ]:
matriz_resultado = np.zeros((N, N, N, N, M), dtype=int)
for var in prob.variables():
    if var.name.startswith("X_") and var.varValue == 1:
        match = re.match(r"X_(\d+)_(\d+)_(\d+)_(\d+)_(\d+)", var.name)
        if match:
            i, j, k, l, m = map(int, match.groups())
            matriz_resultado[i, j, k, l, m] = 1

graficar_escenario(X=matriz_resultado)

In [ ]:
# Imprimir el número de estaciones importantes visitadas
n_est_imp_visitadas_max = sum([I[i,j] * x[indice_en_x[f"Z_{i}_{j}"]].varValue for i,j in estaciones])
print("Número de estaciones importantes visitadas: ", n_est_imp_visitadas_max)

# Imprimir la distancia total recorrida
dist_total_max = sum([DIST[i,j,k,l] * x[indice_en_x[f"X_{i}_{j}_{k}_{l}_{m}"]].varValue for i,j,k,l in conexiones_validas for m in range(M)])
print("Distancia mínima total recorrida: ", dist_total_max)

### SOLUCIONES INTERMEDIAS: MINIMIZACIÓN DE LONGITUD DE VÍAS IMPONIENDO MÍNIMOS DE ESTACIONES ESPECIALES VISITADAS ###

In [ ]:
n_est_imp_visitadas_list = []
dist_total_list = []

for min_est_esp in range(5, 9):

    min_estaciones_especiales = min_est_esp

    rows6, b6 = r6()

    A_leq = vstack([rows1, rows2, rows3, rows4, rows5, rows6, rows7, rows8, rows9, rows10, rows11, rows14, rows18, rows19, rows20], format='csr')
    b_leq = np.hstack([b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b14, b18, b19, b20])

    # === 3. Crear el modelo ===
    prob = pulp.LpProblem("Optimización_MILP", pulp.LpMinimize)

    # === 4. Definir la función objetivo ===
    prob += pulp.lpSum(DIST[i, j, k, l] * x[indice_en_x[f"X_{i}_{j}_{k}_{l}_{m}"]] for i, j, k, l in conexiones_validas for m in range(M))

    # === 5. Especificar las restricciones ===
    for idx in range(A_leq.shape[0]):
        fila = A_leq.getrow(idx)
        prob += pulp.lpSum(fila.data[k] * x[int(fila.indices[k])] for k in range(len(fila.indices))) <= b_leq[idx]

    for idx in range(A_eq.shape[0]):
        fila = A_eq.getrow(idx)
        prob += pulp.lpSum(fila.data[k] * x[int(fila.indices[k])] for k in range(len(fila.indices))) == b_eq[idx]

    # === 6. Resolver ===
    prob.solve()

    # === 7. Obtener resultados ===
    if pulp.LpStatus[prob.status] == 'Optimal':
        print("Solución óptima encontrada:")
        for var in prob.variables():
            if var.varValue > 0:
                print(f"{var.name} = {var.varValue}")  
        print("\n Valor objetivo:", pulp.value(prob.objective))
    else:
        print("No se encontró solución:", pulp.LpStatus[prob.status])


    matriz_resultado = np.zeros((N, N, N, N, M), dtype=int)
    for var in prob.variables():
        if var.name.startswith("X_") and var.varValue == 1:
            match = re.match(r"X_(\d+)_(\d+)_(\d+)_(\d+)_(\d+)", var.name)
            if match:
                i, j, k, l, m = map(int, match.groups())
                matriz_resultado[i, j, k, l, m] = 1

    graficar_escenario(X=matriz_resultado)


    # Imprimir el número de estaciones importantes visitadas
    n_est_imp_visitadas = sum([I[i,j] * x[indice_en_x[f"Z_{i}_{j}"]].varValue for i,j in estaciones])
    print("Número de estaciones importantes visitadas: ", n_est_imp_visitadas)
    n_est_imp_visitadas_list.append(n_est_imp_visitadas)

    # Imprimir la distancia total recorrida
    dist_total = sum([DIST[i,j,k,l] * x[indice_en_x[f"X_{i}_{j}_{k}_{l}_{m}"]].varValue for i,j,k,l in conexiones_validas for m in range(M)])
    print("Distancia mínima total recorrida: ", dist_total)
    dist_total_list.append(dist_total)


# **FRONTERA DE PARETO** 

#### LONGITUD DE VÍA FRENTE A NÚMERO DE ESTACIONES ESPECIALES ####

In [ ]:
# Valores objetivo
x = [n_est_imp_visitadas_min] + n_est_imp_visitadas_list + [n_est_imp_visitadas_max]
y = [dist_total_min] + dist_total_list + [dist_total_max]

# Crear la gráfica
plt.figure(figsize=(8, 6))
plt.plot(x, y, color='violet', linestyle='-', linewidth=2,zorder=1)  # Línea de Pareto
plt.scatter(x, y, color='#c71585', s=100, zorder=2)  # Puntos de Pareto

# Títulos y etiquetas
plt.title('Frontera de Pareto')
plt.xlabel('Nº estaciones especiales')
plt.ylabel('Longitud de vía (km)')

# Mostrar
plt.grid(True)
plt.tight_layout()
plt.show()


#### COSTE FRENTE A NÚMERO DE ESTACIONES ESPECIALES ####

In [ ]:
# Conversión (82 millones por km de vía)
y_cost = [valor * 82 for valor in y]

# Crear la gráfica
plt.figure(figsize=(8, 6))
plt.plot(x, y_cost, color='cornflowerblue', linestyle='-', linewidth=2, zorder=1)  # Línea azul clara
plt.scatter(x, y_cost, color='midnightblue', s=100, zorder=2)  # Puntos azul oscuro


# Títulos y etiquetas
plt.title('Frontera de Pareto')
plt.xlabel('Nº estaciones especiales')
plt.ylabel('Coste estimado (millones de €)')

#  Mostrar
plt.grid(True)
plt.tick_params()
plt.tight_layout()
plt.show()